# HPO AUMCdb: obs_days=7

CQL Hyperparameter Optimization on AUMCdb with:
- obs_days: 7
- grid_step_hours: 8
- inclusion: kdigo2

External validation: AUMCdb → MIMIC

In [ ]:
# ============================================================
# CONFIGURATION - ADJUST THESE VALUES
# ============================================================
OBS_DAYS = 7
GRID_HOURS = 8
INCLUSION = 'kdigo2'
DATABASE = 'aumc'  # Train on AUMCdb

# External validation
EXT_SOURCE = 'aumc'
EXT_TARGET = 'mimic'

# Reward settings
MORTALITY_PENALTY = 'zero'  # Options: 'no', 'zero', 'penalty', 'both'
DISCOUNT = 1.0

# Algorithms to run
RUN_CQL = True
RUN_DDQN = True
RUN_BCQ = False
RUN_NFQ = False

In [ ]:
# 1. GPU Check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 2. Install dependencies
!pip install d3rlpy optuna pandas matplotlib seaborn pyyaml scikit-learn tqdm -q
print("Dependencies installed!")

In [ ]:
# 3. Clone repository
!git clone https://github.com/karel2324/Thesis.git
%cd Thesis/RRT_start/2_Total

In [ ]:
# 4. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 5. Apply configuration
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Dataset config
config['dataset']['obs_days'] = OBS_DAYS
config['dataset']['grid_step_hours'] = GRID_HOURS
config['dataset']['inclusion'] = INCLUSION

# Reward config
config['reward']['mortality_penalty'] = MORTALITY_PENALTY
config['reward']['discount'] = DISCOUNT

# Database selection for each step
db_config = {
    'aumc': DATABASE == 'aumc',
    'mimic': DATABASE == 'mimic'
}

# Ablation study - train on selected database
config['ablation']['databases'] = db_config.copy()

# HPO - train on selected database
config['hpo']['algorithms'] = {
    'cql': RUN_CQL,
    'ddqn': RUN_DDQN,
    'bcq': RUN_BCQ,
    'nfq': RUN_NFQ
}
config['hpo']['databases'] = db_config.copy()

# Behavior cloning - train on selected database
config['behavior_cloning']['databases'] = db_config.copy()

# Evaluation - evaluate on selected database
config['evaluation']['databases'] = db_config.copy()

# External validation - source to target
config['external_validation']['source_database'] = EXT_SOURCE
config['external_validation']['target_database'] = EXT_TARGET

with open('config.yaml', 'w') as f:
    yaml.dump(config, f)

# Print summary
db_name = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
print(f"Config: {db_name} obs{OBS_DAYS} grid{GRID_HOURS} {INCLUSION}")
print(f"  obs_days: {OBS_DAYS}")
print(f"  grid_step_hours: {GRID_HOURS}")
print(f"  inclusion: {INCLUSION}")
print(f"  mortality_penalty: {MORTALITY_PENALTY}")
print(f"  discount: {DISCOUNT}")
print(f"  algorithms: CQL={RUN_CQL}, DDQN={RUN_DDQN}, BCQ={RUN_BCQ}, NFQ={RUN_NFQ}")
print(f"\nDatabase configuration:")
print(f"  ablation_study: {DATABASE}")
print(f"  hpo: {DATABASE}")
print(f"  behavior_cloning: {DATABASE}")
print(f"  evaluation: {DATABASE}")
print(f"  external_validation: {EXT_SOURCE} -> {EXT_TARGET}")

In [ ]:
# 6. Verify data exists
db_prefix = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
data_path = f"/content/drive/MyDrive/Thesis/Data/derived_{db_prefix}_obs{OBS_DAYS}_grid{GRID_HOURS}_{INCLUSION}/"
!ls -la {data_path}

In [ ]:
# 7. Run HPO
!python _7_hpo.py

In [ ]:
# 8. Run Behavior Cloning
!python _8_behavior_cloning.py

In [ ]:
# 9. Run Evaluation
!python _9_evaluation.py

In [ ]:
# 10. Run External Validation (AUMCdb -> MIMIC)
!python _10_external_validation.py

In [ ]:
# 11. Check results
db_prefix = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
results_path = f"/content/drive/MyDrive/Thesis/Data/derived_{db_prefix}_obs{OBS_DAYS}_grid{GRID_HOURS}_{INCLUSION}/Reward_*/"
!ls -la {results_path}